In [1]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token is valid.
Your token has been saved to C:\Users\82109\.huggingface\token
Login successful


In [2]:
from datasets import load_dataset, DatasetDict

Zeroth_Korean = DatasetDict()

Zeroth_Korean["train"] = load_dataset("Bingsu/zeroth-korean", split="train", use_auth_token=True)
Zeroth_Korean["test"] = load_dataset("Bingsu/zeroth-korean", split="test", use_auth_token=True)

print(Zeroth_Korean)

Using custom data configuration Bingsu--zeroth-korean-787ca68963c66467
Found cached dataset parquet (C:/Users/82109/.cache/huggingface/datasets/Bingsu___parquet/Bingsu--zeroth-korean-787ca68963c66467/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Using custom data configuration Bingsu--zeroth-korean-787ca68963c66467
Found cached dataset parquet (C:/Users/82109/.cache/huggingface/datasets/Bingsu___parquet/Bingsu--zeroth-korean-787ca68963c66467/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 22263
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 457
    })
})


In [3]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")

In [4]:
print(Zeroth_Korean["train"][0])

{'audio': {'path': None, 'array': array([-3.05175781e-05,  0.00000000e+00, -3.05175781e-05, ...,
        0.00000000e+00,  0.00000000e+00, -6.10351562e-05]), 'sampling_rate': 16000}, 'text': '인사를 결정하는 과정에서 당 지도부가 우 원내대표 및 원내지도부와 충분한 상의를 거치지 않은 채 일방적으로 인사를 했다는 불만도 원내지도부를 중심으로 흘러나왔다'}


In [5]:
def dataset_prep(batch):
    # load audio
    audio = batch["audio"]

    # calculate log-Mel input features from input audio array
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = processor.tokenizer(batch["text"]).input_ids

    return batch

In [6]:
# applying data preparation func to all training examples using .map
Zeroth_Korean = Zeroth_Korean.map(dataset_prep, remove_columns=Zeroth_Korean.column_names["train"], num_proc=1)

Loading cached processed dataset at C:\Users\82109\.cache\huggingface\datasets\Bingsu___parquet\Bingsu--zeroth-korean-787ca68963c66467\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-bdbe6a7f930a6b10.arrow


  0%|          | 0/457 [00:00<?, ?ex/s]

In [7]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

         # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [8]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [9]:
import evaluate

metric = evaluate.load("wer")

In [10]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [11]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [12]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [21]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-KR",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

PyTorch: setting up devices


In [22]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=Zeroth_Korean["train"],
    eval_dataset=Zeroth_Korean["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

C:\Users\82109\Desktop\Projects\Anarchy\deepnote\FineTuneWhisper-KR\./whisper-small-KR is already a clone of https://huggingface.co/byoussef/whisper-small-KR. Make sure you pull the latest changes with `repo.git_pull()`.
max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


In [23]:
processor.save_pretrained(training_args.output_dir)

Feature extractor saved in ./whisper-small-KR\preprocessor_config.json
tokenizer config file saved in ./whisper-small-KR\tokenizer_config.json
Special tokens file saved in ./whisper-small-KR\special_tokens_map.json
added tokens file saved in ./whisper-small-KR\added_tokens.json


In [24]:
trainer.train()

***** Running training *****
  Num examples = 22263
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4000
  Number of trainable parameters = 241734912
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


RuntimeError: CUDA out of memory. Tried to allocate 824.00 MiB (GPU 0; 6.00 GiB total capacity; 3.92 GiB already allocated; 0 bytes free; 4.70 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
kwargs = {
    "dataset_tags": "Bingsu/zeroth-korean",
    "dataset": "Zeroth-Korean",  # a 'pretty' name for the training dataset
    "dataset_args": "config: kr, split: test",
    "language": "kr",
    "model_name": "Whisper Small KR - BYoussef",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}